## Introduction

DARTS is a scikit-learn kind of open source library with amazing set of time series modelling tools packaged into it. Its easy to use and you can test and compare multiple models at the same time.


For detailed documentation: https://unit8co.github.io/darts/index.html

![](https://unit8.com/wp-content/uploads/2021/07/1_yaHh5V0AgxEYxJYrv9VQmA.png)

In [ ]:
##Installing darts
# !pip install darts

If you are installing it on your own system, you might get an error while building wheel for prophet or C++ compiler. Check on StackOverflow for the relevant solution

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime 

In [ ]:
##Import data
data = pd.read_csv('/kaggle/input/pbd-load-history/PDB_Load_History.csv')
data.head()

In [ ]:
data.info()

In [ ]:
#Converting date column to Datetime
data['date'] = pd.to_datetime(data['date'])
data = data.set_index("date")
data[['demand','temperature']].describe()

## Exploratory Data Analysis

In [ ]:
# plt.rcParams.update(plt.rcParamsDefault)

In [ ]:
plt.style.use("bmh")
plt.figure(figsize=(15,5))
data.demand.plot(label='Demand of electricity')
plt.legend();

Seems like no increasing or decreasing trends

In [ ]:
list(data.columns)

Visualizing demand by year, month, weekday, day, hour

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(data=data, x='year', y='demand');

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(data=data, x='month', y='demand');

July/August months shows higher demand for electricity

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(data=data, x='day', y='demand');

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(data=data, x='weekday', y='demand');

Weekends have lower demand compared to weekdays

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(data=data, x='hour', y='demand');

## Data Preprocessing

For modelling with DARTS, you need a time series object. TimeSeries class of DARTS helps us do that.

In [ ]:
#Resampling demand at daily level
daily_demand = pd.DataFrame(data[['demand']].resample('D', origin = 'start').sum())
# daily_demand.reset_index(inplace=True)
daily_demand.head()

In [ ]:
#Creating timeseries object for demand
from darts import TimeSeries
series_demand = TimeSeries.from_dataframe(daily_demand, value_cols='demand')
series_demand

## Model Training

In [ ]:
# Check for seasonality
from darts.utils.statistics import plot_acf, check_seasonality

for m in range(2, 25):
    is_seasonal, mseas = check_seasonality(series_demand, m=m, alpha=0.05)
    if is_seasonal:
        break

print("seasonal? " + str(is_seasonal))
if is_seasonal:
    print('There is seasonality of order {}.'.format(mseas))

plot_acf(series_demand, m=12, alpha=0.05)

Creating Train & Validation series

In [ ]:
train, val = series_demand.split_before(pd.Timestamp("20120101"))
plt.figure(figsize=(15,5))
train.plot(label="training")
val.plot(label="validation")

Training some models and validating

In [ ]:
from darts.models import ExponentialSmoothing, FFT, Prophet, AutoARIMA, Theta
from darts.metrics import mape, r2_score

def eval_model(model):
    start_time = datetime.now() 
    model.fit(train)
    time_elapsed = datetime.now() - start_time
    forecast = model.predict(len(val))
    print("model {} obtains MAPE: {:.2f}% , Training time: {}".format(model, mape(val, forecast), time_elapsed))

eval_model(ExponentialSmoothing())
eval_model(FFT())
eval_model(Prophet())
eval_model(AutoARIMA())
eval_model(Theta())

Based on error metric, Prophet is the best model for our current use case

In [ ]:
model_demand = Prophet()
model_demand.fit(train)

In [ ]:
# use the model to make a forecast
pred_demand = model_demand.predict(len(val))

plt.figure(figsize=(15, 6))
series_demand.plot(label="actual")
pred_demand.plot(label="forecast")
plt.title("MAPE: {}".format(mape(val, pred_demand)))

Backtesting with some historical data

In [ ]:
def display_forecast(pred_series, ts_transformed, forecast_type, start_date=None):
    plt.figure(figsize=(15, 5))
    if start_date:
        ts_transformed = ts_transformed.drop_before(start_date)
    ts_transformed.univariate_component(0).plot(label="actual")
    pred_series.plot(label=("historic " + forecast_type + " forecasts"))
    plt.title(
        "R2: {}".format(r2_score(ts_transformed.univariate_component(0), pred_series))
    )
    plt.legend()

pred_series = model_demand.historical_forecasts(
    series_demand,
    start=pd.Timestamp("20140901"),
    forecast_horizon=30,
    stride=5,
    verbose=True,
)
display_forecast(pred_series, series_demand, "30 day", start_date=pd.Timestamp("20140901"))

In [ ]:
# investigate the residuals in the validation dataset
from darts.utils.statistics import plot_residuals_analysis
resid = pred_demand - val
plot_residuals_analysis(resid);

## Retraining using Temperature as a Covariate

In [ ]:
#Resampling temperature at daily level
daily_temp = pd.DataFrame(data[['temperature']].resample('D', origin = 'start').mean())
daily_temp.head()

In [ ]:
#Creating timeseries object for temperature
from darts import TimeSeries
series_temp = TimeSeries.from_dataframe(daily_temp, value_cols='temperature')
series_temp

Prophet's fit() only supports future covariates

In [ ]:
model_future_covs = Prophet()
model_future_covs.fit(train, future_covariates=series_temp)

In [ ]:
# use the model to make a forecast
pred_demand_covs = model_future_covs.predict(len(val), future_covariates=series_temp)

plt.figure(figsize=(15, 6))
series_demand.plot(label="actual")
pred_demand_covs.plot(label="forecast")
plt.title("MAPE: {}".format(mape(val, pred_demand_covs)))

This means temperature data is not improving the forecast.

In [ ]:
pred_series = model_demand.historical_forecasts(
    series_demand,
    start=pd.Timestamp("20140901"),
    forecast_horizon=30,
    stride=5,
    verbose=True, future_covariates=series_temp
)
display_forecast(pred_series, series_demand, "30 day", start_date=pd.Timestamp("20140901"))

The above was a quick attempt at forecasting using DARTS. I would try to use some regression, deep learning & ensemble models in future. Happy to have your feedback on the above!